# Cortex Compliance AI - Fine-Tuning Notebook

Fine-tunes Mistral-7B for Russian compliance document generation.

## Quick Start:
1. Runtime → Change runtime type → **T4 GPU**
2. Run all cells
3. Enter your HuggingFace token when prompted

In [ ]:
# Step 1: Install dependencies
!pip install -q transformers==4.36.0 accelerate peft bitsandbytes datasets huggingface_hub trl==0.7.4

In [ ]:
# Step 2: Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Step 3: Training data - Russian Compliance Documents
TRAINING_DATA = [
    {"instruction": "Generate a Personal Data Processing Policy for company ООО Ромашка (INN: 7707123456)", "output": "ПОЛИТИКА ОБРАБОТКИ ПЕРСОНАЛЬНЫХ ДАННЫХ\nООО «РОМАШКА»\n\n1. ОБЩИЕ ПОЛОЖЕНИЯ\n1.1. Настоящая Политика определяет позицию ООО «Ромашка» (ИНН: 7707123456) в области обработки персональных данных.\n1.2. Политика разработана в соответствии с Федеральным законом № 152-ФЗ «О персональных данных».\n\n2. ЦЕЛИ ОБРАБОТКИ\n- осуществление трудовых отношений;\n- выполнение договорных обязательств;\n- обеспечение безопасности.\n\nГенеральный директор _________________ / _______________ /"},
    {"instruction": "Create a consent form for processing personal data in Russian", "output": "СОГЛАСИЕ на обработку персональных данных\n\nЯ, _________________________________________________ (ФИО),\nпаспорт серия _______ № ____________,\n\nдаю согласие на обработку моих персональных данных, включающих:\n- фамилия, имя, отчество;\n- паспортные данные;\n- контактный телефон;\n- адрес электронной почты.\n\nСогласие действует до его отзыва.\n\n«___» ____________ 20__ г.  ___________________ / _____________________ /"},
    {"instruction": "Generate ISPDN description for HR system", "output": "ОПИСАНИЕ ИНФОРМАЦИОННОЙ СИСТЕМЫ ПЕРСОНАЛЬНЫХ ДАННЫХ «Кадры»\n\n1. Наименование: «Кадры»\n2. Назначение: Автоматизация кадрового учета\n\n3. Категории ПДн: ФИО, паспортные данные, ИНН, СНИЛС\n\n4. Уровень защищенности: УЗ-4 (согласно ПП РФ № 1119)\n\n5. Средства защиты:\n- Антивирусное ПО\n- Система контроля доступа\n- Межсетевой экран"},
    {"instruction": "Create Roskomnadzor notification", "output": "УВЕДОМЛЕНИЕ об обработке персональных данных\n\nВ Управление Роскомнадзора\n\nВ соответствии со статьей 22 ФЗ № 152-ФЗ уведомляем о намерении осуществлять обработку персональных данных.\n\nЦель обработки: кадровый учет, договорные отношения\nКатегории ПДн: общие, иные\nСпособ обработки: автоматизированный\n\nРуководитель _________________ / _____________________ /"},
    {"instruction": "Generate DPO appointment order", "output": "ПРИКАЗ о назначении ответственного за обработку персональных данных\n\nВ соответствии со статьей 22.1 ФЗ № 152-ФЗ\n\nПРИКАЗЫВАЮ:\n\n1. Назначить ответственным за организацию обработки персональных данных ___________________\n\n2. Ответственному:\n- организовать работу по обработке ПДн;\n- осуществлять внутренний контроль;\n- обеспечить взаимодействие с Роскомнадзором.\n\nГенеральный директор _________________ / _____________________ /"},
    {"instruction": "Generate KII categorization act for power grid", "output": "АКТ категорирования объекта КИИ\n\nНа основании ФЗ № 187-ФЗ и ПП РФ № 127 проведено категорирование:\n\nОбъект: АСУ ТП\nСфера: Топливно-энергетический комплекс\n\nПоказатели значимости:\n- Социальная: ВЫСОКАЯ\n- Экономическая: ВЫСОКАЯ\n\nРЕШЕНИЕ: Категория значимости ПЕРВАЯ (I)\n\nПредседатель комиссии _________________ / _____________________ /"},
    {"instruction": "Create bank security policy according to GOST 57580", "output": "ПОЛИТИКА ИНФОРМАЦИОННОЙ БЕЗОПАСНОСТИ\n(согласно ГОСТ Р 57580.1-2017)\n\n1. ОБЩИЕ ПОЛОЖЕНИЯ\nПолитика разработана в соответствии с ГОСТ Р 57580.1-2017 и Положениями Банка России.\n\n2. УРОВЕНЬ ЗАЩИТЫ: Уровень 2 (стандартный)\n\n3. ЦЕЛИ:\n- защита информационных активов;\n- обеспечение непрерывности операций;\n- соблюдение требований регуляторов.\n\nНачальник службы ИБ _________________ / _____________________ /"},
    {"instruction": "Explain УЗ-3 protection level according to FSTEC", "output": "УРОВЕНЬ ЗАЩИЩЕННОСТИ 3 (УЗ-3) согласно Приказу ФСТЭК № 21\n\nПрименяется при:\n- Тип угроз 3 + специальные ПДн до 100 000 субъектов\n- Тип угроз 2 + иные ПДн до 100 000 субъектов\n\nОбязательные меры:\n- Идентификация и аутентификация\n- Антивирусная защита\n- Регистрация событий\n- Резервное копирование\n\nТребуются СЗИ классов: СВТ-6, МЭ-5, СОВ-6"}
]

print(f"Training examples: {len(TRAINING_DATA)}")

In [ ]:
# Step 4: Prepare dataset
from datasets import Dataset

def format_prompt(example):
    return {"text": f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"}

dataset = Dataset.from_list(TRAINING_DATA)
dataset = dataset.map(format_prompt)
print(dataset)

In [ ]:
# Step 5: Load model with 4-bit quantization
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
print(f"Model loaded: {MODEL_NAME}")

In [ ]:
# Step 6: Configure LoRA
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Step 7: Train using Trainer (compatible with all versions)
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

training_args = TrainingArguments(
    output_dir="./cortex-compliance-ai",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("Starting training...")
trainer.train()
print("Training complete!")

In [ ]:
# Step 8: Save to Hugging Face Hub
HF_USERNAME = "cortexgrc"
MODEL_REPO = f"{HF_USERNAME}/cortex-compliance-ai"

model.push_to_hub(MODEL_REPO)
tokenizer.push_to_hub(MODEL_REPO)
print(f"Model saved to: https://huggingface.co/{MODEL_REPO}")

In [ ]:
# Step 9: Test the model
test_prompt = "### Instruction:\nGenerate a Personal Data Processing Policy for ООО Тест (INN: 1234567890)\n\n### Response:\n"
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=300,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Done!

Your model is now available at: https://huggingface.co/cortexgrc/cortex-compliance-ai